# Milestone Project 2 - Blackjack Game

## Problem Statement

Create a Complete BlackJack Card Game in Python.

### Requirements

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

### Implementation Style

* OOP and classes in Python

## Design Process

### Game Card
* A game card is printed with a box dash. The card rank is printed as well
* All dealer's cards are hidden except the first card

### Classes, Methods, global Variables
* Create global variables to track:
  * current player
  * card suit, rank and Value
* Create a Card, Deck, and Bet classes

### Card Values
* All face cards (**Jack (J), Queen (Q), King (K)**) count as 10.
* Ace counts as 1 or 11 according to player choice
  * An ace can turn to 1 when
    * the game winner needs to be determined and a player sum exceeds 21 but has an ace among their cards
      * In this case, if the sum will be less than 21 with ace  becomes 1. 
      * This is a valid operation and they player has therefore not **BUST**
* All other cards count for their number 2 to 10.

### Determine number of decks to play with
  
### Player Options
* A player has the following options:
  * Split
    * A split occurs if the first two cards have the same value.
    * The player has the option to split or perform other actions
    * If a player decides to split
      * The two cards are split into separate bets
      * Each split has its own bet and the player can perform independent actions on each split
    * Note:
      * Multiple split can occur within a split 
  * Stand
    * A player **Stand** means the player has decided to take no more cards.
      * Note:
        * When a player stands, the dealer must reveal their cards.
          * If the dealer cannot take a stand yet, the dealer must continue to hit until the dealer hits a stand
            * This means the sum of dealers can must be at least 17.
  * Hit
    * If a player decides to **HIT**, the player must add an extra card to their cards
      * Note:
        * If a player card sum exceeds 21 after a **HIT**, that player has BUST. 
          * This means the dealer automatically wins the round!
  * Double
    * If a player decides to Double, this means the player doubles their wager
    * Next, the player must **HIT**. 
    * Finally, the game condition will be checked and a winner will be determined.
      * Note:
        * **Dealer Stand** must take effect. 

### Game States
* BlackJack
  * If a player has a total of 21 from the first two cards, that is a **BlackJack**.
    * This means this player has automatically won the round
* Push
  * A push occurs when the dealer and player have the total card sum
  * The player and dealer gets to keep their wager
* Bust
  * A bust occurs when one of the players exceed 21
  * A bust occurs for a player has a higher score than the dealer 
  * Loser loses their bet.
* Win
  * A win occurs when a player has a lower card sum than the dealer
  * A win can also occur if a dealer or player **BUST**
  * The winners get to keep the total wager
  * Note:
    * If both players exceed a total of 21, the player with the lower score gets 
* Dealer Stand
  * A dealer must relieve their hidden cards when a player decided to **STAND**.
  * Note:
    * After the dealer relieves their hidden cards,
    * If the dealer's sum is less than 17
      * the dealer must hit until a sum of at least 17 has been reached 
    * Once this condition is satisfied:
      * Check game condition and determine the winner


### Continue Games
* If a game status has been determined, Ask player if they would like to continue the game?

### Reset game
- Reset deck

In [698]:
# Global Variable
import random
from abc import ABC, abstractmethod

suits = ('spades', 'hearts', 'diamonds', 'clubs')
ranks = ('ace','two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'jack', 'queen', 'king')
values = {'two': 2, 'three': 3,'four': 4, 'five': 5, 'six':6, 'seven':7, 'eight':8, 'nine':9, 'ten':10, 'jack': 10, 'queen': 10, 'king': 10, 'ace': 11}
card_emojis = {'spades': '♤', 'hearts': '♡', 'diamonds': '♢', 'clubs': '♣️'}
card_letters = {'two': '2', 'three': '3','four': '4', 'five': '5', 'six':'6', 'seven':'7', 'eight':'8', 'nine':'9', 'ten':'10', 'jack': 'J', 'queen': 'Q', 'king': 'K', 'ace': 'A'}

In [699]:
class Card:
    # constructor
    def __init__(self, suit, rank) -> None:
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        self.emoji = card_emojis[suit]
        self.letter = card_letters[rank]

    # getters and setters
    def getValue(self) -> int:
        return self.value
    
    def getLetter(self) -> int:
        return self.letter

    def printCard(self) -> str:
        print("*"*15)
        if len(str(self.letter)) == 1:
            print("* {letter}           *".format(letter=self.letter))
        else:
            print("* {letter}          *".format(letter=self.letter))
        print("* {emoji}           *".format(emoji=self.emoji))
        print("*             *")
        print("*      {emoji}      *".format(emoji=self.emoji))
        print("*             *")
        print("*           {emoji} *".format(emoji=self.emoji))
        if len(str(self.letter)) == 1:
            print("*           {letter} *".format(letter=self.letter))
        else:
            print("*          {letter} *".format(letter=self.letter))
        print("*"*15)
    
    def __str__(self) -> str:
        return "{rank} of {suit}".format(rank=self.rank, suit=self.suit)

In [701]:
# Deck Class
class Deck:
    # constructor
    def __init__(self, num_cards = 1):
        # Populate each deck with all cards
        self.cards = []
        for suit in suits:
            for rank in ranks:
                self.cards.append(Card(suit, rank))
        # double the size of the deck based on the number of cards provided
        for num in range(1, num_cards):
            self.cards += self.cards

    # Functions

    # shuffle a deck
    def shuffle_deck(self) -> None:
        random.shuffle(self.cards)

    # Deal one card from the deck
    def deal_card(self):
        return self.cards.pop()
    
    def getCards(self):
        return self.cards
        
    # Helper Methods
    def __len__(self) -> int:
        return len(self.cards)
        
    def __str__(self) -> str:
        return "The deck has {total} cards".format(total=len(self))

In [702]:
# Create Chip | Bet class
class Chip:
    def __init__(self, amount=200) -> None:
        self.amount = amount
        self.total = amount
        # track how much the player has lost or won
        self.totalWon = 0
        self.totalBet = 0

    def initialAmount(self) -> int:
        return self.amount

    def balance(self) -> int:
        return self.total

    def withdraw(self, amount) -> None:
        # Edge case: you can only bet if amount is <= total balance
        if(amount > self.total):
            print('Insufficient balance! Request amount ${amount} exceeds current balance ${balance}!'.format(amount=amount, balance = self.total))
        else:
            self.total -= amount
            self.totalBet += amount
            print('Successfully deposited ${amount}. New balance is ${balance}!'.format(amount=amount, balance = self.total))

    def deposit(self, amount) -> None:
        if(amount < 0):
            print('Failed! Only positive transactions are allowed')
        else:
            # update total balance and the totalWon
            self.total += amount
            self.totalWon += amount
            print('Successfully credited ${amount}. New balance is ${balance}!'.format(amount=amount, balance = self.total))
            
    def getTotalBet(self) -> int:
        return self.totalBet

    def getTotalWon(self) -> int:
        return self.totalWon

    def __str__(self) -> str:
        return "Total: ${total}.\nTotal won: ${total_won}.\nTotal bet: ${total_bet}".format(total=self.total, total_won=self.totalWon, total_bet=self.totalBet)


In [703]:
# Create a base class for all types of players
class Actor:
    # constructor
    # create a container for player cards, playerCardTotal, 
    def __init__(self, name, amount) -> None:
        self.name = name
        self.chips = Chip(amount=amount)
        self.cards = []
        self.cardsTotal = 0
        # self.bust = False
        # self.blackjack = False
        self.hidden = False

    
        
    # member functions
    def add_card(self, card) -> None:
        # Add card and update card score
        self.cards.append(card)
        self.cardsTotal += card.getValue()

        # edge case => adjust card totals if greater than 21 and has an ace present
        self.adjustCardTotals()

    def revealCards(self):
        # card start
        s = ""
        for card in self.cards:
            s +="\t"+ "*"*15
        print(s)

        # card value - first
        s = ""
        for card in self.cards:
            if self.hidden:
                s +="\t* {letter}           *".format(letter='?')
            else:
                if len(str(card.letter)) == 1:
                    s += "\t* {letter}           *".format(letter=card.letter)
                else:
                    s += "\t* {letter}          *".format(letter=card.letter)
        print(s)

        
        # card emoji - first
        s = ""
        for card in self.cards:
            if self.hidden:
                s +="\t* {emoji}           *".format(emoji='?')
            else:
                s +="\t* {emoji}           *".format(emoji='?')
        print(s)

        # card space - first
        s = ""
        for card in self.cards:
                s +="\t*             *"
        print(s)

        # card emoji - mid
        s = ""
        for card in self.cards:
            if self.hidden:
                s +="\t*      {emoji}      *".format(emoji='?')
            else:
                s +="\t*      {emoji}      *".format(emoji=card.emoji)
        print(s)

        # card space - last
        s = ""
        for card in self.cards:
                s +="\t*             *"
        print(s)


        # card emoji - last
        s = ""
        for card in self.cards:
            if self.hidden:
                s +="\t*           {emoji} *".format(emoji='?')
            else:
                s +="\t*           {emoji} *".format(emoji=card.emoji)
        print(s)

        # card value - last
        s = ""
        for card in self.cards:
            if self.hidden:
                s +="\t*           {letter} *".format(letter='?')
            else:
                if len(str(card.letter)) == 1:
                    s += "\t*           {letter} *".format(letter=card.letter)
                else:
                    s += "\t*          {letter} *".format(letter=card.letter)
        print(s)
        
        # card end
        s = ""
        for card in self.cards:
            s +="\t"+ "*"*15
        print(s)


    def hasBlackjack(self) -> bool:
        # A blackjack can only occur with the first two cards
        if len(self.cards) == 2:
            return self.cardsTotal == 21

    def hasBust(self):
        # edge case => total exceeds 21 => set bust to true
        return self.cardsTotal > 21
    
    def hasAce(self):
        for card in self.cards:
            return card.rank == 'ace'
    
    def adjustCardTotals(self):
        if self.hasAce():
            if self.cardsTotal > 21:
                self.cardsTotal -= 10

    def getName(self) -> str:
        return self.name
    
    def getCardsTotal(self) -> int:
        return self.cardsTotal

    # def getBust(self) -> bool:
    #     return self.bust
    
    # def getBlackjack(self) -> bool:
    #     return self.blackjack
    
    # def setBust(self, value) -> None:
    #     self.bust = value
    
    # def setBlackjack(self, value) -> None:
    #     self.blackjack = value

    def getHidden(self):
        return self.hiddenCards
    
    def setHidden(self, value):
        self.hidden = value
    
    def __str__(self) -> str:
        return "{name}'s stats.\nInitial AmountCard total: {card_total}.\nCards: {cards}.\nTotal balance: {balance}".format(name=self.name, card_total=self.cardsTotal,cards=self.cards,balance=self.chips.balance() )
    

In [ ]:
class Dealer(Actor):
    # constructor
    def __init__(self, name, amount=10000):
        super().__init__(name, amount)
        # create property to track if dealers cards are hidden
        self.setHidden(True)
        
    
    # # abstract functions
    # def add_card(self, card):
    #     # edge case => check for black jack before adding any cards
    #     # if self.hasBlackjack():
    #     #     self.setBlackjack(True)
    #     #     return
    
    #     self.cards.append(card)
    #     self.cardsTotal += card.getValue()
        
    #     # edge case => adjust card totals if greater than 21 and has an ace present
    #     self.adjustCardTotals()

        
    #     # if self.cardsTotal > 21:
    #     #     self.setBust(True)

    # member functions
    def match_bet(self, amount) -> None:
        # A dealer matches a player bet if a player wins
        self.chips.withdraw(amount=amount)

    def win_bet(self, amount) -> None:
        # a dealer wins a bet if a player Lost
        self.chips.deposit(amount=amount)

    def dealer_stand(self, deck) -> None:
        # edgecase: when dealer takes a stand => All dealer cards must be revealed
        self.setHidden(False)
        # while cardTotal is less than 17
        while self.cardsTotal < 17:
            self.add_card(deck.deal_one())


    # Create a function to properties 

In [705]:
class Player(Actor):
    # constructor
    def __init__(self,name, amount=1000):
        super().__init__(name, amount)
        # self.split = False
        self.cardSplits = []
    
    # abstract functions
    # def add_card(self, card):
    #     # edge case => check for black jack before adding any cards
    #     # if self.hasBlackjack():
    #     #     self.setBlackjack(True)
    #     #     return
        
       
    #     # if self.hasSplit():
    #     #     self.setSplit(True)
    #     #     return

    #     # Add card and update card score
    #     self.cards.append(card)
    #     self.cardsTotal += card.getValue()

    #     # edge case => adjust card totals if greater than 21 and has an ace present
    #     self.adjustCardTotals()

    #     # edge case => check for bust 
    #     # if self.cardsTotal > 21:
    #     #     self.setBust(True)
    
    # member functions
    def hasSplit(self):
        # edge case => check for spilt,  a split occurs when the first two cards are face cards => total == 20
        if len(self.cards) == 2:
            return self.cardsTotal == 20
    
    # def getSplit(self) -> bool:
    #     return self.split
    
    # def setSplit(self, value) -> None:
    #     self.split = value

    def place_bet(self, amount) -> None:
        print(self.cardsTotal)

    def credit_bet(self, amount) -> None:
        print(self.cardsTotal)

    # player actions
    def split() -> None:
        pass

    def double() -> None:
        pass

    def stand() -> None:
        pass


    def hit(self, deck) -> None:
        # Add a card from the deck when a player hits
        self.add_card(deck.deal_one())
        pass

In [ ]:
deck = Deck()
deck.shuffle_deck()
dealer = Dealer(name='dealer')
player = Player(name='player1')
print(dealer)
dealer.add_card(card=deck.deal_card())
dealer.add_card(card=deck.deal_card())
dealer.add_card(card=deck.deal_card())
dealer.revealCards()
dealer.getCardsTotal()


In [ ]:
# Initialize game
# create deck, create player, create dealer

In [ ]:
def welcome_message():
    print('Welcome to blackjack game!')
    print('This blackjack is between one player and an automated dealer!')
    print('The rules are: ')
    print("\t A player can perform the following actions:")
    print("\t\tStand")
    print("\t\t\tThis means the player will no longer pick a card and the dealer is required to reveal their hidden cards. After which the winner is determined!")
    print("\t\tHit")
    print("\t\t\tThis means a player can will pick an extra card. Note: As long as the player's total does not exceed 21, the player can continue to HIT. If sum of cards is greater than 21. The player has BUST.")
    print("\t\tDouble")
    print("\t\t\tA double means the player double their current wager and must HIT. After a hit, the dealer must reveal their cards. The winner is the player with the closest sum of cards to 21.")
    print("\t\tSplit")
    print("\t\t\t.....more details coming")
    print("\t The game states or conditions are:")
    print("\t\tBust")
    print("\t\t\tA bust occurs when a player has a total greater than 21. This implies that player has lost the round!")
    print("\t\tPush")
    print("\t\t\tA push occurs when both players have the same sum of cards. In this case, the player gets to keep their bet.")
    print("\t\tBlackjack")
    print("\t\t\tA blackjack occurs if a player's first two cards equal 21. In this case, the player has the round!. The player wins all the bet!")
    print("Enjoy!")

In [ ]:
def place_bet(player):
    bet = input('Enter player bet: ')
    if bet.isalpha():
        print('Only numbers are allowed!')
    else:
        bet = int(bet)
        print('{player} placed {amount} bet!'.format(player=player.getName(),amount=bet))
        # place requested bet for the current player

In [ ]:
def game_options():
    while True:
        # edge case => if the current player is the dealer, skip
        current_player = 'player'
        if current_player == 'dealer':
            # dealer cannot perform any actions
            break
        # edge case => if has split provide split option
        hasSplit = False
        if hasSplit:
            option = input("Select one of the options: Hit (H), Stand (S), Double (D), Split(SP)?")
        else:
            option = input("Select one of the options: Hit (H), Stand (S), Double (D)?")

        option = option.lower()
        if option.isalpha() != False or option not in "'sp'shd":
            print('Invalid option!')
            continue
        else:
            if option == 'h':
                # perform hit operation on the player
                pass

            if option == 'd':
                # perform double operation on the player
                pass

            if option == 'sp':
                # perform option split operation on the player
                pass

            if option == 's':
                # perform option stand operation on the player
                pass



In [1]:
# After a bet as been placed, we want to print the bet and the num of cards available in the deck
# pass the deck and player 
def display_stats(deck, player):
    print("\t====================================================")
    print("\tnum of cards: {num_cards".format(num_cards=len(deck)))
    print("\t{player}'s bet: ${bet}".format(player=player.getName(), bet=500))
    print("\t====================================================")

In [ ]:
# display the game board
def display_cards():
    ## print dealer's cards => one card shown and other hidden
    print("Display dealer's cards")
    # edge case => we want to reveal dealer cards only when requested => or are not hidden
    # else hide all cards except 1.
    # display dealer's stats board
    print("============================================")
    print("{player}'s cards total: {cards_total}".format(player="dealer", cards_total=8))
    print("Number of cards: {cards_count}".format(cards_count=2))
    print("============================================")
    print("\n")
    # print players cards => one card shown and other hidden
    print("Display player's card")
    # display players stats board    
    print("============================================")
    print("{player}'s cards total: {cards_total}".format(player="player", cards_total=12))
    print("Number of cards: {cards_count}".format(cards_count=2))
    print("Current Balance: ${balance}".format(balance=500))
    print("============================================")
    pass

In [ ]:
def continue_game():
    while True:
        game_status = input('Would you like to continue, (y/n)? ')
        game_status = game_status.lower()
        if game_status.isalpha() != False or game_status not in 'yn':
            print('Invalid selection. Enter y or n!')
            continue
        else:
            if game_status == 'n':
                print('Quitting. Loading game stats....')
                # print game stats
                # game_stats()
            else:
                print('Game on!!')
                # continue game
            # exit loop loop after condition has been matched
            break

In [707]:
# At the end of the game, we want to display the player stats
def game_stats():
    # edge case => if player decides to walk away 
    # edge case => when player goes broke => player.total == 0
    print("============================================")
    print("Initial Amount: {amount}".format(amount=1000))
    # print number of bets won
    print("Number of bets won: {won}".format(won=12))
    # print amount won
    print("Amount won: ${total_won}".format(total_won=1200))
    # print number of bets lost
    print("Number of bets lost: {lost}".format(lost=3))
    # print amount lost
    print("Amount lost: ${total_lost}".format(total_lost=300))
    print("============================================")

In [708]:
# Game setup